In [141]:
# importa pandas para analisis de datos, numpy para operaciones matematicas y requests para solicitudes HTTP
import pandas as pd 
import numpy as np 
import requests

In [142]:
# define la URL de la fuente de datos en formato JSON de actividades y eventos en Madrid
url_api = "https://datos.madrid.es/egob/catalogo/300107-0-agenda-actividades-eventos.json"

In [83]:
# realiza una solicitud GET a la URL y verificamos el codigo de estado de la respuesta
response = requests.get(url_api)
response.status_code

200

In [84]:
# convierte la respuesta en formato JSON a un diccionario de Python
data = response.json() 
data

{'@context': {'c': 'http://www.w3.org/2002/12/cal#',
  'dcterms': 'http://purl.org/dc/terms/',
  'geo': 'http://www.w3.org/2003/01/geo/wgs84_pos#',
  'loc': 'http://purl.org/ctic/infraestructuras/localizacion#',
  'org': 'http://purl.org/ctic/infraestructuras/organizacion#',
  'vcard': 'http://www.w3.org/2006/vcard/ns#',
  'schema': 'https://schema.org/',
  'title': 'vcard:fn',
  'id': 'dcterms:identifier',
  'relation': 'dcterms:relation',
  'references': 'dcterms:references',
  'address': 'vcard:adr',
  'area': 'loc:barrio',
  'district': 'loc:distrito',
  'locality': 'vcard:locality',
  'postal-code': 'vcard:postal-code',
  'street-address': 'vcard:street-address',
  'location': 'vcard:geo',
  'latitude': 'geo:lat',
  'longitude': 'geo:long',
  'organization': 'vcard:org',
  'organization-desc': 'dcterms:description',
  'accesibility': 'org:accesibilidad',
  'services': 'org:servicios',
  'schedule': 'org:horario',
  'organization-name': 'vcard:organization-name',
  'description': '

In [85]:
# imprime las claves principales del JSON para inspeccionarlas
print(data.keys())

dict_keys(['@context', '@graph'])


In [86]:
# obtiene las claves del primer elemento dentro de la lista del atributo '@graph' en el JSON
data['@graph'][0].keys()

dict_keys(['@id', '@type', 'id', 'title', 'description', 'free', 'price', 'dtstart', 'dtend', 'time', 'excluded-days', 'recurrence', 'uid', 'link', 'event-location', 'references', 'relation', 'address', 'location', 'organization'])

In [87]:
# extrae el valor del campo 'title' del primer elemento dentro de la lista '@graph' del JSON
titulo = data.get('@graph')[0].get('title')
titulo

'25º aniversario de la revista La Fragua'

In [88]:
# extrae el codigo postal del campo 'postal-code' dentro de 'area' de la direccion del primer evento
cod_postal = data.get('@graph')[0].get('address').get('area').get('postal-code')
cod_postal

'28005'

In [89]:
# convierte la fecha en formato string a un objeto datetime de pandas
fecha = data.get('@graph')[0].get('dtstart')
fecha_convertida = pd.to_datetime(fecha)
fecha_convertida

Timestamp('2025-02-24 00:00:00')

 El json obtenido es un diccionario que cuenta con dos claves principales, que son '@context', '@graph'. Nosotros necesitamos acceder al valor de la clave '@graph', que es una lista formada por un único elemento, que es un diccionario. Dentro de este diccionario encontramos ya diversas claves de donde vamos a obtener valores como por ejemplo el nombre del evento que se va a obtener de 'title'. Por otro lado, en algunos casos, habrá que acceder al valor de otras claves para poder obtener la información que queremos. Por ejemplo, para la organizacion habrá que acceder en primer lugar al valor de la clave 'organization', que es un diccionario, y sobre ese diccionario acceder al valor de la clave 'organization_name'. También ocurre con el codigo postal pos ejemplo. En estos caso hay que acceder en primer lugar al valor de la clave 'address', que es otro diccionario y dentro de este diccionario acceder al valor de la clave 'area' que es otro diccionario, y por último, dentro de este último diccionario acceder al valor de la clave 'postal-code'. Es ir navegando por diccionarios hasta lograr obtener toda la información deseada. 

 Por otro lado, como queremos obtener los eventos en cuyas fechas de inicio y fin de evento se icnluyen las fechas de estancia (1 y 2 de marzo), vamos a traernos las fechas de inicio y fin de estancia del archivo parquet para poder filtrar con ellas. Aunque la estancia como tal sea de un día (el 1 de marzo), también se van a atraer los eventos del 2 de marzo, pues al final a los hoteles les interesa poder ofrecer lo mejor, y puede ser interesante que como cliente aunque el día 2 termines tu estancia en el hotel, como sigues en Madrid, poder acudir a algún evento y aprovechar un día más en la ciudad.

In [90]:
# lee el archivo parquet y muestra una muestra aleatoria de 5 filas del dataframe
df_raw = pd.read_parquet("../data/reservas_hoteles.parquet")
df_raw.sample(5)

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
6377,a44901fe-5cb2-4bfb-9efe-4f6b1c2b5425,f9f006da-ceaa-422f-9494-ad2be44a4ecc,Teodora,Mendez,teodora.mendez@example.com,False,2025-02-08,2025-03-01,2025-03-02,40,415.18,Hotel Rincón Sereno,3.0,Madrid
38,166f113f-64cf-408d-b1c3-30ef155f8049,d0de4a2b-eb08-4dc6-aa8d-1efd2f007539,Ezequiel,Cuesta,ezequiel.cuesta@example.com,False,2025-02-10,2025-03-01,2025-03-02,48,314.73,Hotel Costa Azul,5.0,Madrid
14045,e2a2eaa0-20f8-4a11-a3c7-e974f3ba1165,5804bc48-99d0-40cb-b0ea-4e3d133bd6fd,Elías,Echeverría,elías.echeverría@example.com,False,2025-02-06,2025-03-01,2025-03-02,27,354.17,Hotel Jardines del Rey,3.0,Madrid
7904,4105a4c0-382b-4d06-aabc-50be7fc541c0,892037e4-bf31-4b1e-a0f2-aae94e6dc6a4,Noemí,Llorente,noemí.llorente@example.com,False,2025-02-01,2025-03-01,2025-03-02,3,224.20,Hotel Puerta del Cielo,3.0,Madrid
1100,401e5b50-378a-4078-934a-bc6d0d519ada,c307e92a-d16c-49a6-9758-34704e20a00d,Salvador,Bayón,salvador.bayón@example.com,True,,2025-03-01,2025-03-02,186,NaN,,NaN,


In [140]:
# convierte las fechas de "final_estancia" e "inicio_estancia" a formato datetime
final_estancia = pd.to_datetime(df_raw["final_estancia"][0])
final_estancia_correcto = final_estancia.replace(hour=23, minute=59, second=0, microsecond=0) # para incluir tambien el dia 2 de marzo en los eventos 
inicio_estancia = pd.to_datetime(df_raw["inicio_estancia"][0])
print(final_estancia_correcto)
print(inicio_estancia)

2025-03-02 23:59:00
2025-03-01 00:00:00


In [143]:
# define la funcion scrap_info_eventos que extrae la informacion de los eventos desde una api
def scrap_info_eventos(url):

    response = requests.get(url)
    data = response.json() 

    # inicializa un diccionario vacio para almacenar la informacion de los eventos
    info_eventos = {
        "nombre_evento": [],
        "url_evento": [],
        "codigo_postal": [],
        "direccion": [],
        "horario": [],
        "organizacion": [],
        "inicio_evento": [],
        "fin_evento": [],
        "ciudad": []
    }

    # itera sobre cada evento en la lista "@graph" del JSON
    for evento in data["@graph"]:
        # convierte las fechas de inicio y fin del evento a tipo datetime
        start_date = pd.to_datetime(evento.get('dtstart'))
        end_date = pd.to_datetime(evento.get('dtend'))

        if start_date <= pd.to_datetime("2025-03-02 23:59:00") and end_date >= pd.to_datetime("2025-03-01 00:00:00"): # verifica si el evento ocurre dentro del rango de fechas deseado
            # extrae informacion del evento con valores por defecto np.nan si no se encuentra y la va añadiendo al diccionario 
            info_eventos["nombre_evento"].append(evento.get('title', None))
            info_eventos["url_evento"].append(evento.get('link', None))
            address = evento.get("address", {})
            area = address.get("area", {}) 
            info_eventos["codigo_postal"].append(area.get("postal-code", None))
            info_eventos["direccion"].append(area.get("street-address", None))
            info_eventos["horario"].append(evento.get("time", "") or None)
            organizacion = evento.get('organization', {})
            info_eventos["organizacion"].append(organizacion.get('organization-name', None))
            info_eventos["inicio_evento"].append(evento.get("dtstart", None).split(" ")[0] if evento.get("dtstart") else None)
            info_eventos["fin_evento"].append(evento.get("dtend", None).split(" ")[0] if evento.get("dtend") else None)
            info_eventos["ciudad"].append("Madrid")

    # retorna el diccionario con la informacion de los eventos
    return info_eventos

In [144]:
# llama a la funcion de scraping, convierte el diccionario en un DataFrame y lo muestra
dictio_final = scrap_info_eventos(url_api)
df_eventos = pd.DataFrame(dictio_final)
df_eventos

,nombre_evento,url_evento,codigo_postal,direccion,horario,organizacion,inicio_evento,fin_evento,ciudad
0,25º aniversario de la revista La Fragua,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28005,CALLE SAN JUSTO 5,None,Biblioteca Pública Municipal Iván de Vargas (C...,2025-02-24,2025-03-02,Madrid
1,60 Premio Reina Sofía de Pintura y Escultura,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28009,PASEO COLOMBIA 1,None,Centro Cultural Casa de Vacas (Retiro),2025-02-27,2025-03-23,Madrid
2,A toda vela,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045,PLAZA LEGAZPI 8,None,Matadero Madrid,2024-10-01,2025-06-01,Madrid
3,Acompañamiento digital a personas mayores,http://www.madrid.es/sites/v/index.jsp?vgnextc...,None,None,None,None,2024-10-14,2025-06-30,Madrid
4,Acompañar en la pérdida,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28029,AVENIDA MONFORTE DE LEMOS 38,17:00,Biblioteca Pública Municipal José Saramago (Fu...,2025-01-14,2025-06-17,Madrid
...,...,...,...,...,...,...,...,...,...
270,"Y tú, ¿cómo viajas?",http://www.madrid.es/sites/v/index.jsp?vgnextc...,28037,CALLE HERMANOS GARCIA NOBLEJAS 14,10:00,Biblioteca Pública Municipal Ciudad Lineal (Ci...,2024-12-04,2025-05-28,Madrid
271,Yoga para el buen trato II,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045,PASEO CHOPERA 10,18:00,Espacio de Igualdad Juana Doña. Arganzuela,2025-02-18,2025-03-11,Madrid
272,'Lina en el mundo de Paul Klee' por Alejandra ...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28029,AVENIDA MONFORTE DE LEMOS 38,08:30,Biblioteca Pública Municipal José Saramago (Fu...,2025-02-04,2025-03-25,Madrid
273,'Refugiados' por Myriam Laguía,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28019,CALLE ISAAC ALBENIZ 1,19:00,Biblioteca Pública Municipal Ana María Matute ...,2025-02-14,2025-03-13,Madrid


In [138]:
# guarda el DataFrame como un archivo pickle 
df_eventos.to_pickle("../data/datos_extraidos/tabla_eventos.pickle")